In [1]:
## install packages
!python3 -m pip install -q "mxnet<2.0.0"
!python3 -m pip install -q autogluon
!python3 -m pip install -q -U graphviz
!python3 -m pip install -q -U scikit-learn

In [3]:
from autogluon.tabular import TabularDataset, TabularPredictor
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
#EDA和特征工程需要用到的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy
import scipy.stats as stats
from scipy.stats import norm,skew
import statsmodels.api as sm
import warnings
import math
#建模需要用到的库
import shap
from catboost import Pool
from catboost import CatBoostRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

%matplotlib inline
warnings.simplefilter('ignore')

plt.rcParams['font.sans-serif']=['SimHei'] #显示中文标签
plt.rcParams['axes.unicode_minus']=False

In [4]:
train=TabularDataset('/kaggle/input/housing-price111/train.csv')
test=TabularDataset('/kaggle/input/housing-price111/test.csv')

In [5]:
#去除outlier
sns.boxplot(train['房屋租金'])
plt.title("Box Plot before outlier removing")
plt.show()
def drop_outliers(df, field_name):
    df.drop(df[df[field_name]>7000].index, inplace=True)
    df.drop(df[df[field_name]<45].index, inplace=True)
drop_outliers(train,'房屋租金')
sns.boxplot(train['房屋租金'])
plt.title("Box Plot after outlier removing")
plt.show()

In [6]:
#分离目标和特征
target = train['房屋租金']
test_id = test['ID']
test = test.drop(['ID'],axis = 1)
train_df = train.drop(['房屋租金','ID'], axis = 1)

#合并训练集与测试集
train_test = pd.concat([train_df,test], axis=0, sort=False)

In [7]:
#（1）去除掉无用的列 (缺失值太多加上相关系数太小)
useless = ['电力基础价格','没有停车位','上传日期'] 
train_test = train_test.drop(useless, axis = 1)
#(2)
#train_test['可带宠物'].fillna('negotiable',inplace=True)
#train_test['最后翻新年份'].fillna(train_test['建成年份'],inplace=True)

In [8]:
#将bool值和object值转为字符并编码
train_test['有阳台'] = train_test['有阳台'].astype('category')
train_test['有厨房'] = train_test['有厨房'].astype('category')
train_test['有地窖'] = train_test['有地窖'].astype('category')
train_test['有电梯'] = train_test['有电梯'].astype('category')
train_test['有花园'] = train_test['有花园'].astype('category')
train_test['是新建筑'] = train_test['是新建筑'].astype('category')
train_test['可带宠物'] = train_test['可带宠物'].astype('category')

In [9]:
#特征工程
train_test['居住面积--2']=train_test['居住面积']**2
train_test['每间房间平均面积']=train_test['居住面积']/(train_test['房间数量']+1)
train_test['每平方米服务费']=train_test['服务费']/(train_test['居住面积']+1)
train_test['每平方米供暖费用']=train_test['供暖费用']/(train_test['居住面积']+1)
room_features1 = ['有阳台','有厨房','有地窖','有电梯','有花园','是新建筑']
train_test["roomFeatures1"] = train_test[room_features1].sum(axis=1)
train_test["不同建成年份平均服务费"] = train_test.groupby("建成年份")["服务费"].transform("mean")

train_test["区域2平均建成年份"] = train_test.groupby("区域2")["建成年份"].transform("median")
train_test["区域1平均建成年份"] = train_test.groupby("区域1")["建成年份"].transform("median")
train_test["区域3平均建成年份"] = train_test.groupby("区域3")["建成年份"].transform("median")
train_test["区域1平均服务费"] = train_test.groupby("区域1")["服务费"].transform("mean")
train_test["区域2平均服务费"] = train_test.groupby("区域2")["服务费"].transform("mean")
train_test["区域3平均服务费"] =train_test.groupby("区域3")["服务费"].transform("mean")
train_test["区域1服务费标准差"] = train_test.groupby("区域1")["服务费"].transform("std")
train_test["区域2服务费标准差"] = train_test.groupby("区域2")["服务费"].transform("std")
train_test["区域3服务费标准差"] = train_test.groupby("区域3")["服务费"].transform("std")
train_test["区域1价格趋势"] = train_test.groupby("区域1")["价格趋势"].transform("mean")
train_test["区域2价格趋势"] = train_test.groupby("区域2")["价格趋势"].transform("mean")
train_test["区域3价格趋势"] = train_test.groupby("区域3")["价格趋势"].transform("mean")
train_test["区域1价格趋势标准差"] = train_test.groupby("区域1")["价格趋势"].transform("std")
train_test["区域2价格趋势标准差"] = train_test.groupby("区域2")["价格趋势"].transform("std")
train_test["区域3价格趋势标准差"] = train_test.groupby("区域3")["价格趋势"].transform("std")
train_test["区域1居住面积"] = train_test.groupby("区域1")["居住面积"].transform("mean")
train_test["区域2居住面积"] = train_test.groupby("区域2")["居住面积"].transform("mean")
train_test["区域3居住面积"] = train_test.groupby("区域3")["居住面积"].transform("mean")
train_test["区域1居住面积标准差"] = train_test.groupby("区域1")["居住面积"].transform("std")
train_test["区域2居住面积标准差"] = train_test.groupby("区域2")["居住面积"].transform("std")
train_test["区域3居住面积标准差"] = train_test.groupby("区域3")["居住面积"].transform("std")
train_test["区域1供暖费用"] = train_test.groupby("区域1")["供暖费用"].transform("mean")
train_test["区域2供暖费用"] = train_test.groupby("区域2")["供暖费用"].transform("mean")
train_test["区域3供暖费用"] = train_test.groupby("区域3")["供暖费用"].transform("mean")
train_test["区域1供暖费用标准差"] = train_test.groupby("区域1")["供暖费用"].transform("std")
train_test["区域2供暖费用标准差"] = train_test.groupby("区域2")["供暖费用"].transform("std")
train_test["区域3供暖费用标准差"] = train_test.groupby("区域3")["供暖费用"].transform("std")
train_test["街道平均建成年份"] = train_test.groupby("街道")["建成年份"].transform("median")
train_test["街道平均服务费"] = train_test.groupby("街道")["服务费"].transform("mean")
train_test["街道价格趋势"] = train_test.groupby("街道")["价格趋势"].transform("mean")
train_test["街道居住面积"] = train_test.groupby("街道")["居住面积"].transform("mean")
train_test["街道供暖费用"] = train_test.groupby("街道")["供暖费用"].transform("mean")
train_test["街道服务费标准差"] = train_test.groupby("街道")["服务费"].transform("std")
train_test["街道供暖费用标准差"] = train_test.groupby("街道")["供暖费用"].transform("std")
train_test["街道居住面积标准差"] = train_test.groupby("街道")["居住面积"].transform("std")
train_test["街道价格趋势标准差"] = train_test.groupby("街道")["价格趋势"].transform("std")


#查看频率
temp = train_test['房屋状况'].value_counts().to_dict()
train_test['房屋状况_counts'] = train_test['房屋状况'].map(temp)
temp = train_test['内饰质量'].value_counts().to_dict()
train_test['内饰质量_counts'] = train_test['内饰质量'].map(temp)
temp = train_test['加热类型'].value_counts().to_dict()
train_test['加热类型_counts'] = train_test['加热类型'].map(temp)
temp = train_test['房间数量'].value_counts().to_dict()
train_test['房间数量_counts'] = train_test['房间数量'].map(temp)
temp = train_test['所处楼层'].value_counts().to_dict()
train_test['所处楼层_counts'] = train_test['所处楼层'].map(temp)
temp = train_test['建筑楼层'].value_counts().to_dict()
train_test['建筑楼层_counts'] = train_test['建筑楼层'].map(temp)
temp = train_test['建成年份'].value_counts().to_dict()
train_test['建成年份_counts'] = train_test['建成年份'].map(temp)
temp = train_test['上传图片数'].value_counts().to_dict()
train_test['上传图片数_counts'] = train_test['上传图片数'].map(temp)
temp = train_test['房屋类型'].value_counts().to_dict()
train_test['房屋类型_counts'] = train_test['房屋类型'].map(temp)
temp = train_test['最后翻新年份'].value_counts().to_dict()
train_test['最后翻新年份_counts'] = train_test['最后翻新年份'].map(temp)

#对分类特征进行编码
train_test_dummy = pd.get_dummies(train_test)
#找出偏度较大的数值特征
numeric_features = ['服务费','供暖费用','居住面积','价格趋势']
skewed_features = train_test_dummy[numeric_features].apply(lambda x: skew(x)).sort_values(ascending=False)
high_skew = skewed_features[skewed_features > 0.5]
skew_index = high_skew.index
#用log变换对偏度大的数值特征做变换
for i in skew_index:
    train_test_dummy[i] = np.log1p(train_test_dummy[i])

In [10]:
#经过对数转换后的房屋租金

target_log = np.log1p(target)

fig, ax = plt.subplots(1,2, figsize= (15,5))
fig.suptitle("qq-plot & distribution SalePrice ", fontsize= 15)

sm.qqplot(target_log, stats.t, distargs=(4,),fit=True, line="45", ax = ax[0])
sns.distplot(target_log, kde = True, hist=True, fit = norm, ax = ax[1])
plt.show()

In [11]:
#训练集和测试集分离

train = train_test_dummy[0:199839]
test = train_test_dummy[199839:]
test['ID'] = test_id

In [12]:
train=pd.concat([train,target_log],axis=1)

In [13]:
train.info(max_cols=150)

In [ ]:

metric = 'mae'  
label ='房屋租金'
predictor = TabularPredictor(label, eval_metric=metric).fit(train,excluded_model_types = ['KNN'],presets='best_quality')

In [ ]:
pred = predictor.predict(test.drop(columns=['ID']))
test['房屋租金'] = pred

In [ ]:
test.info()

In [ ]:
test['房屋租金']=np.exp(test['房屋租金'])

In [ ]:
test[['ID','房屋租金']].to_csv('/kaggle/working/result.csv', index=False)